# Chatbot informativo implementado al 100% en Python

### Es un ejemplo sencillo de chatbot que implementa el corpus en un archivo '.txt' y que emplea las librerías nltk y scikitlearn

#### El chatbot informa a los usuarios acerca de las normas de un crucero. Es un ejemplo básico, pero que bien sirve de ejemplo de uso de lematización y búsqueda de coincidencias entre las preguntas de usuario y las diferentes respuestas posibles mediante el modelo "cosine_similarity"

#### Resumen técnico.

##### 1.- En una variable de texto se almacena el corpus (diferentes respuestas posibles al usuario).
##### 2.- Cuando el usuario plantea una pregunta, se agrega -temporalmente- al final de la lista de respuestas. A todo este contenido se le eliminan signos de puntuación, se tokeniza, lematiza y se extraen sus caracterísaticas -mediante TfidfVectorizer de sklearn-. A partir de ellas y empleando un modelo del tipo "cosine_similarity" se buscan las respuestas más coincidentes con la pregunta del usuario, se elige la que mayor grado de coincidentcia muestra y se responde con ella.
##### 3.- Adicionalmente se ha incluido un pequeño módulo de saludo inicial, que aleatoriamente elige una respuesta entre varias posibles.


#### Próximamente subiré un sistema similar pero del tipo "voice bot", empleando para ello librerías de reconocimiento y síntesis de voz

In [1]:
# Importación de librerías
import nltk
import numpy as np
import random
import string
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from spellchecker import SpellChecker

#nltk.download('punkt') # Instalar módulo punkt si no está ya instalado (solo ejecutar la primera vez)
#nltk.download('wordnet') # Instalar módulo wordnet si no está ya instalado (solo ejecutar la primera vez)



#### 1 Carga del corpus

In [3]:
# Cargar el corpus estructurado
with open('/home/mauricio/repos/source/chatbot/data.json', 'r') as f:
    corpus = json.load(f)

# Inicializar la variable 'raw' con el contenido del corpus
raw = " ".join([item['question'] + " " + item['answer'] for item in corpus['faq']])

#### 2 Definición de funciones y variables de apoyo

In [12]:
raw=raw.lower() # Convertimos todo el texto a minúsculas, para evitar deficiencias en la extracción de características

sent_tokens = nltk.sent_tokenize(raw) # Convierte el corpus a una lista de sentencias
word_tokens = nltk.word_tokenize(raw) # Convierte el corpus a una lista de palabras

lemmer = nltk.stem.WordNetLemmatizer() # Instanciamos el lematizador, con el que convertir las palabras  a sus raíces contextuales

#LemTokens es una función que lematiza todos los tokens que se le pasan como parámetro
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

# remove_punct es un diccionario del tipo (0signo de puntuación', None), que se emplea en la función
# LemNormalize para sustituir los signos de puntuación por "nada" es decir, eliminarlos.
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# Dado un texto como parámetro, elimina los signos de puntuación, lo convierte a minúsculas,
# lo tokeniza -por palabras- y finalmente lo lematiza
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Inicializar el corrector ortográfico
spell = SpellChecker(language='es')

def corregir_entrada(entrada):
    """
    Corrige faltas ortográficas en una entrada de texto.
    Si no se encuentra una corrección, se mantiene la palabra original.
    """
    palabras = entrada.split()
    palabras_corregidas = [spell.correction(palabra) or palabra for palabra in palabras]  # Usar 'or' para manejar None
    return ' '.join(palabras_corregidas)

# Diccionario de sinónimos y términos relacionados para expansión
terminos_ampliados = {
    # Términos relacionados con el proyecto
    "título": ["nombre", "denominación", "tema", "proyecto"],
    "proyecto": ["estudio", "investigación", "trabajo", "iniciativa"],
    "exposición": ["radiación", "emisión", "campos"],
    "electromagnética": ["electromagnetismo", "electromagnético", "radiación", "EMF"],
    "machine learning": ["ml", "aprendizaje automático", "aprendizaje de máquina", "ia", "inteligencia artificial"],
    
    # Términos relacionados con objetivos
    "objetivos": ["metas", "propósitos", "fines", "finalidad"],
    "general": ["principal", "primario", "central"],
    "específicos": ["secundarios", "concretos", "particulares"],
    
    # Términos relacionados con modelos
    "modelos": ["algoritmos", "técnicas", "métodos", "enfoques"],
    "regresión": ["predicción", "estimación"],
    "árboles": ["decision trees", "árbol de decisión"],
    "random forest": ["bosque aleatorio", "rf"],
    "xgboost": ["gradient boosting", "boosting"],
    
    # Términos relacionados con datos
    "datos": ["información", "dataset", "conjunto de datos", "fuentes"],
    "variables": ["características", "features", "atributos", "parámetros"],
    
    # Términos relacionados con el problema
    "problema": ["desafío", "reto", "cuestión", "dificultad"],
    "planteamiento": ["formulación", "definición", "descripción"],
    
    # Términos relacionados con beneficios
    "beneficios": ["ventajas", "utilidad", "provecho", "aportes"],
    "ventajas": ["beneficios", "fortalezas", "puntos fuertes"],
    
    # Términos relacionados con conclusiones
    "conclusiones": ["resultados", "hallazgos", "descubrimientos", "inferencias"]
}

def expandir_terminos(texto):
    """
    Expande los términos clave en el texto con sinónimos y términos relacionados.
    """
    palabras = texto.lower().split()
    texto_expandido = texto.lower()
    
    # Buscar términos clave en el texto
    for termino, expansiones in terminos_ampliados.items():
        if termino in texto.lower():
            # Añadir términos relacionados al texto expandido
            terminos_adicionales = " ".join(expansiones)
            texto_expandido += f" {terminos_adicionales}"
    
    return texto_expandido

# Mejorar la función LemNormalize para usar la expansión de términos
def LemNormalize_mejorado(text):
    """
    Versión mejorada de LemNormalize que también expande términos clave.
    """
    # Expandir términos clave
    texto_expandido = expandir_terminos(text)
    
    # Aplicar la normalización estándar
    return LemTokens(nltk.word_tokenize(texto_expandido.lower().translate(remove_punct_dict)))

# Crear una lista de preguntas del corpus
preguntas_corpus = [item['question'] for item in corpus['faq']]

# Función mejorada para buscar la respuesta más relevante usando similitud semántica

def buscar_respuesta_semantica(user_question, umbral_base=0.28, modo="adaptativo"):
    """
    Encuentra la respuesta más relevante en el corpus usando similitud semántica mejorada.
    
    Parámetros:
    - user_question: Pregunta del usuario
    - umbral_base: Umbral base para considerar una respuesta como relevante
    - modo: Tipo de umbral a utilizar ('fijo', 'adaptativo', 'diferencial', 'hibrido')
    """
    # Preprocesar la pregunta del usuario con expansión de términos
    user_question_expandida = expandir_terminos(user_question)
    user_tokens = LemNormalize_mejorado(user_question_expandida)
    user_question_procesada = ' '.join(user_tokens)  # Convertir lista de palabras a texto
    
    # Preprocesar las preguntas del corpus si no se ha hecho aún
    preguntas_procesadas = []
    for pregunta in preguntas_corpus:
        pregunta_expandida = expandir_terminos(pregunta)
        tokens = LemNormalize_mejorado(pregunta_expandida)
        preguntas_procesadas.append(' '.join(tokens))
        
    # Combinar la pregunta del usuario con las preguntas procesadas del corpus
    todas_preguntas = preguntas_procesadas + [user_question_procesada]
    
    # Vectorizar las preguntas con parámetros mejorados
    vectorizador = TfidfVectorizer(
        min_df=1,                 # Incluir términos que aparecen al menos en 1 documento
        max_df=0.95,              # Excluir términos que aparecen en más del 95% de documentos
        ngram_range=(1, 3),       # Considerar hasta trigramas para capturar frases más largas
        sublinear_tf=True,        # Aplica logaritmo para reducir el impacto de términos frecuentes
        use_idf=True,             # Usar IDF para dar más peso a términos discriminativos
        smooth_idf=True           # Evitar división por cero
    )
    vectores = vectorizador.fit_transform(todas_preguntas)
    
    # Calcular la similitud coseno entre la pregunta del usuario y las preguntas del corpus
    similitudes = cosine_similarity(vectores[-1], vectores[:-1])[0]
    
    # Encontrar el índice de la pregunta más similar y la segunda más similar
    similitudes_ordenadas = np.sort(similitudes)[::-1]
    indice_max = similitudes.argmax()
    max_similitud = similitudes[indice_max]
    
    # Si tenemos al menos dos preguntas para comparar
    segunda_max = similitudes_ordenadas[1] if len(similitudes_ordenadas) > 1 else 0
    diferencia = max_similitud - segunda_max
    
    # Aplicar diferentes estrategias de umbral según el modo
    if modo == "fijo":
        # Umbral fijo simple
        if max_similitud < umbral_base:
            return "Lo siento, no tengo una respuesta para esa pregunta. Por favor, intenta con otra consulta."
    
    elif modo == "adaptativo":
        # Umbral adaptativo basado en la distribución de similitudes
        media_similitud = similitudes.mean()
        desv_similitud = similitudes.std()
        # Si la mejor similitud está cerca de la media, podría ser ruido
        umbral_adaptativo = min(umbral_base, media_similitud + desv_similitud * 0.8)
        
        if max_similitud < umbral_adaptativo:
            return "Lo siento, no tengo una respuesta para esa pregunta. Por favor, intenta con otra consulta."
    
    elif modo == "diferencial":
        # Umbral basado en la diferencia entre la mejor y segunda mejor coincidencia
        # Si hay una gran diferencia, tenemos más confianza
        if len(similitudes_ordenadas) > 1:
            # Reducimos el umbral si hay una gran diferencia (alta confianza)
            umbral_diferencial = umbral_base - min(0.18, diferencia * 1.8)
        else:
            umbral_diferencial = umbral_base
            
        if max_similitud < umbral_diferencial:
            return "Lo siento, no tengo una respuesta para esa pregunta. Por favor, intenta con otra consulta."
    
    elif modo == "hibrido":
        # Combinación de adaptativo y diferencial
        media_similitud = similitudes.mean()
        desv_similitud = similitudes.std()
        
        # Componente adaptativo
        umbral_adaptativo = min(umbral_base, media_similitud + desv_similitud)
        
        # Componente diferencial
        if len(similitudes_ordenadas) > 1:
            umbral_diferencial = umbral_base - min(0.15, diferencia * 1.5)
        else:
            umbral_diferencial = umbral_base
        
        # Tomar el mínimo de ambos (más permisivo)
        umbral_hibrido = min(umbral_adaptativo, umbral_diferencial)
        
        if max_similitud < umbral_hibrido:
            return "Lo siento, no tengo una respuesta para esa pregunta. Por favor, intenta con otra consulta."
    
    # Devolver la respuesta correspondiente con un nivel de confianza
    respuesta = corpus['faq'][indice_max]['answer']
    
    # Incluir nivel de confianza solo si es relevante
    if max_similitud > 0.8:
        return respuesta  # Alta confianza, no mostramos el valor
    elif max_similitud > 0.6:
        return respuesta  # Confianza media, no mostramos el valor
    else:
        # Baja confianza, podríamos mostrar el valor o no
        return respuesta

#### Evaluación de precisión del chatbot

In [13]:
# Actualizar la función evaluar_precision_optimizada

def evaluar_precision_optimizada():
    """
    Evalúa la precisión del chatbot con distintas configuraciones para encontrar la óptima.
    """
    # Configuraciones a probar (ampliadas)
    configuraciones = [
        {"umbral": 0.25, "modo": "fijo"},
        {"umbral": 0.28, "modo": "adaptativo"},
        {"umbral": 0.3, "modo": "adaptativo"},
        {"umbral": 0.35, "modo": "diferencial"},
        {"umbral": 0.32, "modo": "diferencial"},
        {"umbral": 0.28, "modo": "hibrido"}  # Nuevo modo híbrido
    ]
    
    resultados_config = {}
    
    # Crear datos de prueba
    datos_prueba = []
    
    # Usar preguntas originales del corpus
    for item in corpus['faq'][:15]:  # Usar más preguntas para una evaluación más completa
        datos_prueba.append({
            "pregunta": item['question'], 
            "respuesta_esperada": item['answer']
        })
    
    # Incluir las variaciones como antes
    variaciones_preguntas = [
        {"pregunta": "¿Podrías decirme cuál es el título del proyecto?", 
         "respuesta_esperada": corpus['faq'][0]['answer']},
        {"pregunta": "título del proyecto", 
         "respuesta_esperada": corpus['faq'][0]['answer']},
        {"pregunta": "nombre del proyecto de exposición electromagnética", 
         "respuesta_esperada": corpus['faq'][0]['answer']},
        {"pregunta": "objetivos principales", 
         "respuesta_esperada": corpus['faq'][1]['answer']},
        {"pregunta": "¿Me podrías explicar qué es la exposición electromagnética?", 
         "respuesta_esperada": corpus['faq'][2]['answer']},
        {"pregunta": "modelos utilizados en machine learning", 
         "respuesta_esperada": corpus['faq'][3]['answer']},
        {"pregunta": "fuente de los datos del proyecto", 
         "respuesta_esperada": corpus['faq'][4]['answer']},
        {"pregunta": "explicación del problema principal", 
         "respuesta_esperada": corpus['faq'][5]['answer']},
        {"pregunta": "ventajas de este estudio", 
         "respuesta_esperada": corpus['faq'][6]['answer']},
        {"pregunta": "resultados finales del proyecto", 
         "respuesta_esperada": corpus['faq'][7]['answer']}
    ]
    
    # Añadir más variaciones más desafiantes
    variaciones_adicionales = [
        {"pregunta": "dime sobre el título", 
         "respuesta_esperada": corpus['faq'][0]['answer']},
        {"pregunta": "que hace el proyecto", 
         "respuesta_esperada": corpus['faq'][1]['answer']},
        {"pregunta": "campos electromagnéticos explicación", 
         "respuesta_esperada": corpus['faq'][2]['answer']},
        {"pregunta": "técnicas de ML usadas", 
         "respuesta_esperada": corpus['faq'][3]['answer']},
        {"pregunta": "de dónde sacaron la información", 
         "respuesta_esperada": corpus['faq'][4]['answer']}
    ]
    
    # Añadir todas las variaciones
    datos_prueba.extend(variaciones_preguntas)
    datos_prueba.extend(variaciones_adicionales)
    
    # Para cada configuración, evaluar la precisión
    for config in configuraciones:
        umbral = config["umbral"]
        modo = config["modo"]
        
        print(f"\nEvaluando con umbral={umbral}, modo={modo}")
        
        aciertos = 0
        resultados = []
        
        for caso in datos_prueba:
            # Usar la configuración específica
            respuesta_generada = buscar_respuesta_semantica(
                caso['pregunta'], 
                umbral_base=umbral,
                modo=modo
            )
            
            # Comparar respuestas ignorando el nivel de confianza (si está presente)
            respuesta_generada_limpia = respuesta_generada.split(" (Confianza:")[0]
            respuesta_esperada_limpia = caso['respuesta_esperada'].split(" (Confianza:")[0]
            
            es_correcto = respuesta_generada_limpia == respuesta_esperada_limpia
            
            if es_correcto:
                aciertos += 1
                
            resultados.append({
                "pregunta": caso['pregunta'],
                "respuesta_esperada": caso['respuesta_esperada'],
                "respuesta_generada": respuesta_generada,
                "correcto": es_correcto
            })
        
        precision = aciertos / len(datos_prueba)
        resultados_config[f"{modo}_{umbral}"] = precision
        
        # Mostrar resultados
        print(f"Precisión del chatbot: {precision * 100:.2f}%")
        print(f"Aciertos: {aciertos}/{len(datos_prueba)}")
        
        # Mostrar errores (limitados para evitar sobrecarga)
        print("\nDetalles de algunas predicciones incorrectas:")
        errores_mostrados = 0
        for resultado in resultados:
            if not resultado["correcto"] and errores_mostrados < 5:  # Limitar a 5 errores mostrados
                print(f"Pregunta: {resultado['pregunta']}")
                print(f"Esperado: {resultado['respuesta_esperada'][:50]}...")
                print(f"Generado: {resultado['respuesta_generada'][:50]}...")
                print("-" * 50)
                errores_mostrados += 1
    
    # Encontrar la mejor configuración
    mejor_config = max(resultados_config, key=resultados_config.get)
    print(f"\nMejor configuración: {mejor_config} con precisión: {resultados_config[mejor_config] * 100:.2f}%")
    
    return resultados_config, mejor_config

# Ejecutar la evaluación optimizada
resultados_optimizados, mejor_configuracion = evaluar_precision_optimizada()


Evaluando con umbral=0.25, modo=fijo
Precisión del chatbot: 73.33%
Aciertos: 22/30

Detalles de algunas predicciones incorrectas:
Pregunta: nombre del proyecto de exposición electromagnética
Esperado: Predicción de niveles de exposición electromagnéti...
Generado: Es la exposición a campos electromagnéticos genera...
--------------------------------------------------
Pregunta: modelos utilizados en machine learning
Esperado: Se usaron modelos como Regresión Lineal, Árboles d...
Generado: Se utilizaron métricas como el Error Cuadrático Me...
--------------------------------------------------
Pregunta: explicación del problema principal
Esperado: El problema radica en la falta de un modelo predic...
Generado: El problema radica en la falta de un modelo predic...
--------------------------------------------------
Pregunta: resultados finales del proyecto
Esperado: Se demostró que es posible predecir la fluctuación...
Generado: ¿Es posible predecir la fluctuación de los niveles...
-------

#### 3 Preprocesamiento del texto y evaluación de la similitud entre el mensaje de usuario y las respuestas definidas en el corpus

In [14]:
# Función mejorada para determinar la similitud del texto insertado y el corpus
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)
    
    # Utilizar una vectorización más avanzada
    TfidfVec = TfidfVectorizer(
        tokenizer=LemNormalize, 
        stop_words=stopwords.words('spanish'),
        min_df=1,
        max_df=0.9,
        ngram_range=(1, 2),
        sublinear_tf=True
    )
    
    caract_textos = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(caract_textos[-1], caract_textos)
    
    # Obtenemos los índices ordenados por similitud (de menor a mayor)
    indices_ordenados = vals.argsort()[0]
    
    # Tomamos los dos índices más altos (el último y el penúltimo)
    idx_mejor = indices_ordenados[-2]  # -2 porque el último es la propia consulta
    idx_segundo_mejor = indices_ordenados[-3] if len(indices_ordenados) > 2 else indices_ordenados[-2]
    
    # Aplanamos el array para análisis
    flat = vals.flatten()
    flat.sort()
    
    # Calculamos la diferencia entre las dos mejores coincidencias
    mejor_coincidencia = flat[-2]
    segunda_mejor = flat[-3] if len(flat) > 2 else 0
    diferencia = mejor_coincidencia - segunda_mejor
    
    # Aplicamos un umbral adaptativo basado en la diferencia
    umbral_adaptativo = 0.15 - min(0.1, diferencia * 0.5)
    
    if mejor_coincidencia < umbral_adaptativo:
        robo_response = "Lo siento, no te he entendido bien. ¿Podrías reformular tu pregunta? Si necesitas ayuda, escribe 'ayuda'."
    else:
        respuesta = sent_tokens[idx_mejor]
        
        # Si hay una gran diferencia entre la mejor y segunda mejor coincidencia,
        # tenemos más confianza en nuestra respuesta
        if diferencia > 0.2:
            # Alta confianza - usamos la respuesta tal cual
            if len(respuesta) > 200:
                respuesta = respuesta[:200] + '...'
        else:
            # Baja diferencia - posible ambigüedad
            # Intentamos una respuesta híbrida o más conservadora
            if len(respuesta) > 150:
                respuesta = respuesta[:150] + '...'
            
        robo_response = respuesta
    
    sent_tokens.remove(user_response)
    return robo_response

#### 4 Definición de funcionalidades de saludo y despedida

In [15]:
saludo_inputs = ("hola", "buenas", "saludos", "qué tal", "hey", "buenos días", "ayuda")
saludo_outputs = [
    "Hola, ¿cómo puedo ayudarte?",
    "Hola, ¿en qué puedo asistirte?",
    "Hola, dime cómo puedo ayudarte."
]

despedidas = [
    "Nos vemos, espero haberte ayudado.",
    "Hasta pronto, ¡cuídate!",
    "Chao, que tengas un buen día."
]

def saludos(sentence):
    for word in sentence.split():
        if word.lower() in saludo_inputs:
            return random.choice(saludo_outputs)

def despedida():
    return random.choice(despedidas)

#### 5 Bucle conversacional

In [16]:
# Reemplazar la parte final del bucle conversacional

flag = True
print("CHATBOT: Mi nombre es CHATBOT. Contestaré a tus preguntas acerca del proyecto. Si necesitas ayuda, escribe 'ayuda'. Para salir, escribe 'salir'.")

# Extraer la mejor configuración de la evaluación
mejor_config_partes = mejor_configuracion.split('_')
mejor_modo = mejor_config_partes[0]
mejor_umbral = float(mejor_config_partes[1])

print(f"CHATBOT: Usando configuración optimizada: modo={mejor_modo}, umbral={mejor_umbral}")

while flag:
    user_response = input().lower()
    user_response = corregir_entrada(user_response)  # Corregir la entrada del usuario
    
    if user_response != 'salir':
        if user_response in ['gracias', 'muchas gracias']:
            print("CHATBOT: No hay de qué.")
        elif user_response == 'ayuda':
            print("CHATBOT: Puedes preguntarme sobre los participantes, objetivos, conclusiones, modelos, etc. Si quieres salir, escribe 'salir'.")
        elif saludos(user_response) is not None:
            print("CHATBOT: " + saludos(user_response))
        else:
            # Usar la configuración optimizada
            respuesta = buscar_respuesta_semantica(
                user_response, 
                umbral_base=mejor_umbral,
                modo=mejor_modo
            )
            print("CHATBOT: " + respuesta)
    else:
        flag = False
        print("CHATBOT: " + despedida())

CHATBOT: Mi nombre es CHATBOT. Contestaré a tus preguntas acerca del proyecto. Si necesitas ayuda, escribe 'ayuda'. Para salir, escribe 'salir'.
CHATBOT: Usando configuración optimizada: modo=adaptativo, umbral=0.28
CHATBOT: Hola, dime cómo puedo ayudarte.
CHATBOT: Participaron Andres Mauricio Ardila, Claudia Ines Giraldo, Marisela Lotero Zuluaga y como ejecutor técnico, la Ing. Darly Mildred Delgado.
CHATBOT: Predicción de niveles de exposición electromagnética en Colombia utilizando técnicas de Machine Learning.
CHATBOT: Predicción de niveles de exposición electromagnética en Colombia utilizando técnicas de Machine Learning.
CHATBOT: Predicción de niveles de exposición electromagnética en Colombia utilizando técnicas de Machine Learning.
CHATBOT: Predicción de niveles de exposición electromagnética en Colombia utilizando técnicas de Machine Learning.
CHATBOT: Chao, que tengas un buen día.
